In [7]:
# import numpy as np
# import pyaudio
# import time
# import librosa

# class AudioHandler(object):
#     def __init__(self):
#         self.FORMAT = pyaudio.paFloat32
#         self.CHANNELS = 1
#         self.RATE = 44100
#         self.CHUNK = 1024 * 2
#         self.p = None
#         self.stream = None

#     def start(self):
#         self.p = pyaudio.PyAudio()
#         self.stream = self.p.open(format=self.FORMAT,
#                                   channels=self.CHANNELS,
#                                   rate=self.RATE,
#                                   input=True,
#                                   output=False,
#                                   stream_callback=self.callback,
#                                   frames_per_buffer=self.CHUNK)

#     def stop(self):
#         self.stream.close()
#         self.p.terminate()

#     def callback(self, in_data, frame_count, time_info, flag):
#         numpy_array = np.frombuffer(in_data, dtype=np.float32)
#         librosa.feature.mfcc(numpy_array)
#         return None, pyaudio.paContinue

#     def mainloop(self):
#         while (self.stream.is_active()): # if using button you can set self.stream to 0 (self.stream = 0), otherwise you can use a stop condition
#             time.sleep(2.0)


# audio = AudioHandler()
# audio.start()     # open the the stream
# audio.mainloop()  # main operations with librosa
# audio.stop()

In [18]:
import os

import tensorflow as tf
from tensorflow import keras

import tensorflow_io as tfio

In [19]:
import pyaudio 
import struct
import numpy as np

import matplotlib.pyplot as plt

%matplotlib tk

CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000

In [21]:
mel_model = tf.keras.models.load_model('../data/saved_models/mel_1_sec_model_(95acc)/mel_1_sec_model')

# Check its architecture
mel_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resizing_4 (Resizing)        (None, 128, 128, 1)       0         
_________________________________________________________________
normalization_2 (Normalizati (None, 128, 128, 1)       3         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 126, 126, 32)      320       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 246016)           

In [22]:
def get_stft_snapshots(index, stft_dict):
    
    r_id = index['recording_id']

    f_path = get_filepath(r_id)
    
    audio = tfio.audio.AudioIOTensor(f_path)
    sr = audio.rate

#     dur = get_duration(index)
    dur = 1
    
    for i in range(dur):
        
        start = i + 1 * sr * floor(index['t_min'])
        stop = start + sr

        audio_slice = audio[start: stop + 1]
        
        audio_tensor = tf.squeeze(audio_slice, axis=[-1])
        
        tensor = tf.cast(audio_tensor, tf.float32)
        
        spectrogram = tfio.experimental.audio.spectrogram(
            tensor,
            nfft=512,
            window=512,
            stride=256
        )
        
        mel_spectrogram = tfio.experimental.audio.melscale(
            spectrogram, 
            rate=sr, 
            mels=128, 
            fmin=93.75,     # 20, 
            fmax=11627.90) # 15000)
            
        dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
            mel_spectrogram, 
            top_db=80)
        
        
        stft_dict['dbscale_mel'].append(mel_spectrogram.numpy())
        stft_dict['mel'].append(mel_spectrogram.numpy())
        stft_dict['stft'].append(spectrogram.numpy())
        stft_dict['species_id'].append(index['species_id'])
    
    
    return None

In [23]:
def prep_audio(audio_input):
    
    audio = tfio.auido.AudioIOTensor(audio_input)
    sr = audio.rate
    
    audio_tensor = tf.squeeze(audio, axis=[-1])
    
    tensor = tf.cast(audio_tensor, tf.float32)
    
    spectrogram = tfio.experimental.audio.spectrogram(
        tensor,
        nfft=512,
        window=512,
        stride=256)

    mel_spectrogram = tfio.experimental.audio.melscale(
        spectrogram, 
        rate=sr, 
        mels=128, 
        fmin=93.75,     # 20, 
        fmax=11627.90) # 15000)

    dbscale_mel_spectrogram = tfio.experimental.audio.dbscale(
        mel_spectrogram, 
        top_db=80)
    
    
    return dbscale_mel_spectrogram